Getting started

In [4]:
import spotify_enrichment
import pandas as pd
import time


In [2]:
playlists = {
    'playlistid': [
        '37i9dQZF1EIh5QTm0PNBlW', '37i9dQZF1DX4aYNO8X5RpR', '4X4dEJ2dDkT11S8UBR6wTO', '37i9dQZF1EIhkGftn1D0Mh',
        '37i9dQZF1EIhbb05MSdzdC', '37i9dQZF1EIh0gn0qhBsTI', '37i9dQZF1DX1BzILRveYHb', '37i9dQZF1DWYoDXiQsd3D2',
        '0V5IsHm0VJbmeffuLzgoc3', '37i9dQZF1EIgxHuuVqSn9D', '37i9dQZF1EIdChYeHNDfK5', '37i9dQZF1EIdXhgO6P7kXm',
        '70CgUbf2RjYd5eqXYvObpT', '4raqLXnmb8WYkjfed9olAR', '37i9dQZF1DX64iwDddhmfW', '37i9dQZF1EIeXFUANbVlkW',
        '0oDLvP7gYHz4hEd5mBzAE1', '37i9dQZF1EIfxVrvtd70vA', '37i9dQZF1EIeOmlQSSmx93', '37i9dQZF1DX2pSTOxoPbx9',
        '1i4tfEypXvbW8ju1nX3ATA', '2k06m32lCGscZHFzQNgec0', '37i9dQZF1EIgxHuuVqSn9D', '1i9sX2Z7CBREGy1tKPhMWg',
        '4SRaENSFQQEBZctPr1KOAn', '4BPiZjsF2bZIy1VGbpFoOS', '6sHd5BU0W8bgvHcYAGflFO', '37i9dQZF1EIhPBBOvFSqO1',
        '37i9dQZF1EIf68orkYt0JL', '37i9dQZF1EIgLJ4kVgZmvy', '3e1mML0cAxyool80MzIgXJ', '37i9dQZF1DWTUoU0bd0p84',
        '2NjcocrUZCh2il9A2q2wMZ', '37i9dQZF1DX9fZ7amiNVu6', '37i9dQZF1EIfvipBGiKzgO', '7MqGk3dNkwJFR5pEGj8ZA5',
        '348KZxNG6YSsyVDSBFJtPz', '37i9dQZF1EIdweJIvlLS4r', '37i9dQZF1EIgcvK8Ci6YZz', '37i9dQZF1EIhrpDnXo5ST9',
        '5LKwsGYskR7UUHwsBB1dCx', '37i9dQZF1EIhj2isIFa2I5', '37i9dQZF1EIdvkdlmXsh0P', '4raqLXnmb8WYkjfed9olAR'
    ],
    'event': [
        'Rain', 'Rain', 'Rain', 'Sun', 'Sun', 'Sun', 'Sun', 'Cloud', 'Cloud', 'Cloud', 'Cloud', 'Cloud', 'Cloud', 'Snow',
        'Snow', 'Snow', 'Snow', 'Storm', 'Storm', 'Storm', 'Drizzle', 'Drizzle', 'Drizzle', 'Drizzle', 'Drizzle', 'Drizzle',
        'Drizzle', 'Spring', 'Spring', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Fall', 'Fall', 'Fall', 'Winter',
        'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter'
    ],
    'type': [
        'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather',
        'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Weather',
        'Weather', 'Weather', 'Weather', 'Weather', 'Weather', 'Season', 'Season', 'Season', 'Season', 'Season', 'Season',
        'Season', 'Season', 'Season', 'Season', 'Season', 'Season', 'Season', 'Season', 'Season', 'Season', 'Season'
    ]
}

In [14]:
songs = {
    'song': ['good luck babe', 'hot girl bummer', 'happy im', 'turbulent', 'video phone'],
    'artist': ['chappell roan', 'blackbear', 'umi', 'waterparks', 'beyonce']
}

In [6]:
numeric_values = pd.DataFrame()

# Iterate over all playlists
for i in range(len(playlists['playlistid'])):
    print(f"Playlist {i+1}/{len(playlists['playlistid'])}")
    # Get the playlist id, event, and type
    playlist_id = playlists['playlistid'][i]
    event = playlists['event'][i]
    type_ = playlists['type'][i]

    # Get track info for the playlist
    track_info = spotify_enrichment.get_playlist_tracks(playlist_id)
    
    # Check if track_info is not None before proceeding
    if track_info is not None:
        for info in track_info:
            artist, track_title, track_uri, track_id, track_popularity = info
            song_measures = spotify_enrichment.api.tracks.audio_features(track_id)
            numeric_values_dict = {k: v for k, v in song_measures.items() if isinstance(v, (int, float))}
            numeric_values_dict['song'] = track_title
            numeric_values_dict['artist'] = artist
            numeric_values_dict['event'] = event
            numeric_values_dict['type'] = type_
            numeric_values_dict['popularity'] = track_popularity
            numeric_values_df = pd.Series(numeric_values_dict).to_frame().T
            numeric_values = pd.concat([numeric_values, numeric_values_df], ignore_index=True)
            print(f"Song: {track_title} - Artist: {artist}", end='\r', flush=True)
            time.sleep(0.1)  # Optional: add a small delay for better visualization
    else:
        print(f"No track info available for playlist {playlist_id}")

print(numeric_values)

Getting tracks for playlist 37i9dQZF1EIh5QTm0PNBlW
Getting tracks for playlist 37i9dQZF1DX4aYNO8X5RpRlinater RevivalRevival
Getting tracks for playlist 4X4dEJ2dDkT11S8UBR6wTOoture Calma Michaelsaling
Getting tracks for playlist 37i9dQZF1EIhkGftn1D0Mhantntytyfor Cutiekeys
Getting tracks for playlist 37i9dQZF1EIhbb05MSdzdCAndy Grammerpaie Meiers, Claud, Melanie Faye, & HXNS) - Artist: Cautious Clay
Getting tracks for playlist 37i9dQZF1EIh0gn0qhBsTIrumsssellsner
Getting tracks for playlist 37i9dQZF1DX1BzILRveYHb11 - Artist: Queeneadneking Headsin Timberlake


In [12]:
numeric_values.shape

(785, 17)

In [14]:
numeric_values.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song,artist,event,type
0,0.357,0.425,5,-7.301,1,0.0333,0.584,0,0.322,0.27,102.078,198040,3,Heather,Conan Gray,Rain,Weather
1,0.539,0.179,7,-14.33,1,0.0309,0.921,0.393,0.106,0.145,140.01,198027,3,Like Real People Do,Hozier,Rain,Weather
2,0.669,0.613,4,-11.12,0,0.036,0.487,0.176,0.132,0.566,110.668,220720,4,For Emma,Bon Iver,Rain,Weather
3,0.508,0.792,0,-7.311,0,0.0297,0.229,0.124,0.145,0.601,147.067,320467,4,Space Song,Beach House,Rain,Weather
4,0.741,0.697,0,-7.028,1,0.0277,0.0664,0.000023,0.133,0.774,116.109,160133,4,Have You Ever Seen The Rain,Creedence Clearwater Revival,Rain,Weather


In [13]:
numeric_values.to_csv('spotify_data.csv', index=False)

In [4]:
song_measures, song_location = spotify_enrichment.spotify_main()
pd.Series(song_measures).to_frame()

Getting song info for california


,0
danceability,0.454
energy,0.363
key,1
loudness,-10.037
mode,1
speechiness,0.0585
acousticness,0.566
instrumentalness,0
liveness,0.0788
valence,0.361


In [12]:
numeric_values = {k: v for k, v in song_measures.items() if isinstance(v, (int, float))}
numeric_values = pd.Series(numeric_values).to_frame().T
numeric_values

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.454,0.363,1.0,-10.037,1.0,0.0585,0.566,0.0,0.0788,0.361,129.974,198089.0,4.0


In [4]:
pd.DataFrame(
    spotify_enrichment.get_playlist_tracks('2k06m32lCGscZHFzQNgec0'), 
    columns=['track', 'artist', 'uri', 'playlistid', 'popularity']
    )

Getting tracks for playlist 2k06m32lCGscZHFzQNgec0


,song,artist,uri,playlistid,popularity
0,Bon Iver,Rosyln,spotify:track:4k7x3QKrc3h3U0Viqk0uop,4k7x3QKrc3h3U0Viqk0uop,76
1,Bon Iver,Blood Bank,spotify:track:15WbjBy1bzH1UsUldBwwlW,15WbjBy1bzH1UsUldBwwlW,61
2,Bon Iver,Flume,spotify:track:0bhBtXHQX3u6dSAPF59qGx,0bhBtXHQX3u6dSAPF59qGx,61
3,Coldplay,Sparks,spotify:track:7D0RhFcb3CrfPuTJ0obrod,7D0RhFcb3CrfPuTJ0obrod,85
4,Liana Flores,rises the moon,spotify:track:51Grh1RyUDcMBbpuyUIUHI,51Grh1RyUDcMBbpuyUIUHI,82
5,Lizzy McAlpine,ceilings,spotify:track:3hiuZQQ33aDnu4Xmmn8Lr7,3hiuZQQ33aDnu4Xmmn8Lr7,0
6,Alison Sudol,It's Really Raining,spotify:track:1CaE0HSFy3ACitYvvGsh3h,1CaE0HSFy3ACitYvvGsh3h,25
7,Bon Iver,Beach Baby,spotify:track:2LXtyj95dK56ENS1QLt4IB,2LXtyj95dK56ENS1QLt4IB,75
8,Gregory Alan Isakov,Big Black Car,spotify:track:3Kj2EWpIBnvETsYq4cq0IH,3Kj2EWpIBnvETsYq4cq0IH,75
9,The Japanese House,i saw you in a dream,spotify:track:0ltmioOsLQsL0OFgcPbdVi,0ltmioOsLQsL0OFgcPbdVi,54
